# Install Libraries

%pip install ortools folium -q
%pip install haversine -q
%pip install geopy -q
%pip install seaborn -q

## import libraries

In [1]:
import seaborn as sns
import numpy as np
import requests
import io
import os
import math
import math     
import pandas as pd 
import pytest
import matplotlib.pyplot as plt
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import io
import requests
import folium
from folium.plugins import FastMarkerCluster
from scipy.spatial import ConvexHull
from folium import Tooltip
import numpy as np
from functools import partial
import collections
import datetime
import csv
import math
from geopy import Nominatim
from IPython.display import display, HTML

# Function Definitions

In [2]:
import datetime
R = 6371.0
EARTH_RADIUS = 6371.0
import pandas as pd
import openpyxl
from openpyxl.styles import Font, PatternFill

def write_ouput_files(dataset):
    # Import necessary libraries
    import pandas as pd
    import openpyxl
    from tqdm import tqdm
    from openpyxl.styles import Font, PatternFill

    # Reload the original dataset
    df = dataset

    # Initialize a dictionary to hold the structured data for the route schedule
    route_schedule = {}
    # Populate the route_schedule dictionary based on the DataFrame
    for _, row in df.iterrows():
        new_rt = row['NEW RT']
        new_day = row['NEW DAY']
        city = row['CITY']
        cust_name = row['CUST NAME']
        new_stop = row['NEW STOP']
        machine = row['Machine']  # Updated to match the case in the dataset
        
        if new_rt not in route_schedule:
            route_schedule[new_rt] = {}
        
        if new_day not in route_schedule[new_rt]:
            route_schedule[new_rt][new_day] = {'cities': set(), 'customers': []}
        
        route_schedule[new_rt][new_day]['cities'].add(city)
        route_schedule[new_rt][new_day]['customers'].append({'name': cust_name, 'stop': new_stop, 'machine': machine})
    # Check the structure by showing data for one route
    next(iter(route_schedule.items()))
    # Initialize a new Excel workbook and remove the default sheet
    wb = openpyxl.Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)

    # Function to fill in the customer details for each day and stop
    def fill_customer_details(sheet, row, col, day, stop, route_data):
        data = route_data.get(day, {}).get('customers', [])
        for customer in data:
            if customer['stop'] == stop:
                sheet.cell(row=row, column=col, value=stop)
                sheet.cell(row=row, column=col+1, value=customer['name'])
                sheet.cell(row=row, column=col+2, value=customer['machine'])
                return
        sheet.cell(row=row, column=col, value="")
        sheet.cell(row=row, column=col+1, value="")
        sheet.cell(row=row, column=col+2, value="")

    # Loop through each unique NEW RT
    for route in tqdm(route_schedule.keys()):
        ws = wb.create_sheet(title=str(route))
        # Initialize row and column pointers
        row = 1
        col = 1
        # Initialize day counter for column headers
        day_counter = 1
        while day_counter <= 25:  # There will never be more than 25 days
            # Fill in the column headers for the days and customer details
            ws.cell(row=row, column=col, value=f"Day {day_counter}")
            ws.cell(row=row+1, column=col, value="Stop")
            ws.cell(row=row+1, column=col+1, value="Customer")
            ws.cell(row=row+1, column=col+2, value="Machine")
            # Fill in customer details for each stop (1-10)
            for stop in range(1, 11):  # There will never be more than 10 stops
                fill_customer_details(ws, row=row+1+stop, col=col, day=day_counter, stop=stop, route_data=route_schedule[route])  
            # Update the column pointer to skip to the next day block (4 columns per day + 1 empty column)
            col += 4
            day_counter += 1 
            # Check if we've filled in 5 days, if so, skip a row and reset column pointer
            if (day_counter - 1) % 5 == 0:
                row += 13  # Skip to next block of rows (10 stops + 2 header rows + 1 empty row)
                col = 1

    # Font and Fill styles
    bold_font = Font(bold=True)
    italic_font = Font(italic=True)
    blue_fill = PatternFill(start_color="00CCFF", end_color="00CCFF", fill_type="solid")


    # Loop through each unique NEW RT
    for route in route_schedule.keys():
        ws = wb.create_sheet(title=str(route+1))
        # Initialize row and column pointers
        row = 1
        col = 1  
        # Initialize day counter for column headers
        day_counter = 1
        while day_counter <= 25:  # There will never be more than 25 days
            # Fill in the column headers for the days and customer details
            cell = ws.cell(row=row, column=col+1, value=f"Day {day_counter}")
            cell.font = bold_font  # Make it bold
            # Update header names and styles
            ws.cell(row=row+1, column=col, value="Stop").font = italic_font
            ws.cell(row=row+1, column=col+1, value="Customer").font = italic_font
            ws.cell(row=row+1, column=col+2, value="Sales").font = italic_font  # Changed from "Machine" to "Sales"
            # Apply light blue background to headers
            for c in range(col, col+3):
                ws.cell(row=row+1, column=c).fill = blue_fill

            # Fill in customer details for each stop (1-10)
            for stop in range(1, 11):  # There will never be more than 10 stops
                fill_customer_details(ws, row=row+1+stop, col=col, day=day_counter, stop=stop, route_data=route_schedule[route])
            
            # Update the column pointer to skip to the next day block (3 columns per day, removed empty column)
            col += 3
            day_counter += 1
            
            # Check if we've filled in 5 days, if so, skip a row and reset column pointer
            if (day_counter - 1) % 5 == 0:
                row += 13  # Skip to next block of rows (10 stops + 2 header rows + 1 empty row)
                col = 1  # Reset to column B
    #create workbook file name that includes the date and time
    now = datetime.datetime.now()
    date_time_str = now.strftime("%m%d%Y_%H%M%S")

    # Save the Excel workbook
    wb.save(f'./data/Route_Schedule_Formatted_{date_time_str}.xlsx')
    #save dataset to a csv file for future use
    dataset.to_csv(f'./data/reroute_dataset_{date_time_str}.csv')
def print_starts_ends(data,dataset):
    for start in data['starts']:
        crint(dataset.iloc[start]['CUST NAME'],"white", "dark grey", "italic")
    for endd in data['ends']:
        crint(dataset.iloc[endd]['CUST NAME'],"white", "dark grey", "italic")
def best_tracker(solution, prev_obj, best_obj):
    crint(f"""The current solution was completed in {solution.ObjectiveValue()} km. ""","white", "dark grey", "bold_italic")
    crint(f"""The previous best solution was completed in {best_obj} km.""","white", "dark grey", "bold_italic")
    crint(f"The most previous solution was completed in {prev_obj} km.","white", "dark grey", "bold_italic")
    if best_obj > solution.ObjectiveValue():
        best_obj = solution.ObjectiveValue()
        crint(f"Therefore, we have a new best solution with a value of {str(best_obj)} km", "red", "yellow", "bold_italic")
    else:    
        if prev_obj > solution.ObjectiveValue():
            crint(f"Therefore, we have solution that is better than the previous by {prev_obj - solution.ObjectiveValue()} km","white", "dark grey", "bold_italic")
        else:
            crint(f"Therefore, the current solution is  {solution.ObjectiveValue()-prev_obj} km longer","white", "dark grey", "bold_italic")
    prev_obj = solution.ObjectiveValue()
    return prev_obj, best_obj
def get_new_stop_count(dataset):
    # Get the number of stops from each route
    stop_count = dataset.groupby('Vehicle')['NEW DAY'].nunique()
    return stop_count
import networkx as nx
from haversine import haversine, Unit
from os import write
# Define a function to calculate the distance between two nodes
def calculate_distance(node1, node2):
    lat1, lon1 = node1['Latitude'], node1['Longitude']
    lat2, lon2 = node2['Latitude'], node2['Longitude']
    return haversine((lat1, lon1), (lat2, lon2))
def split_into_days(path, graph, min_stops, max_stops):
    days = []
    while len(path) > min_stops:
        max_distance = 0
        split_point = min(max_stops, len(path))
        for i in range(min_stops, min(max_stops, len(path) - 1)):  # ensure we don't go out of range
            distance = graph[path[i]][path[i+1]]['weight']
            if distance > max_distance:
                max_distance = distance
                split_point = i + 1
        days.append(path[:split_point])
        path = path[split_point:]
    if path:
        days.append(path)
    return days
def set_days_for_each_route(dataset, min_stops=5, max_stops=8):
    unique_routes = dataset['Vehicle'].unique()

    branch = 650
    # Iterate over unique routes
    for route in unique_routes:
        # Subset your data for the current route
        subset_data = dataset[dataset['Vehicle'] == route]
        # Create a graph for this route
        graph = nx.Graph()
        # Add nodes to the graph
        for index, row in subset_data.iterrows():
            graph.add_node(index, Latitude=row['Latitude'], Longitude=row['Longitude'])
        # Add edges to the graph (complete graph, every node is connected to every other node)
        for node1, data1 in graph.nodes(data=True):
            for node2, data2 in graph.nodes(data=True):
                if node1 != node2:
                    distance = calculate_distance(data1, data2)
                    graph.add_edge(node1, node2, weight=distance)

        # Find the shortest path in the graph
        # Note: This assumes that the depot is the first node in the subset_data DataFrame
        depot = subset_data.index[0]
        path = nx.approximation.christofides(graph)#, source=depot)

        # Split the path into days
        days = split_into_days(path, graph,min_stops, max_stops)
        # Assign the order of the path and the day to the 'DAY' and 'DAY_ORDER' columns of the subset_data DataFrame
        day_order = 0
    
        # print(f"Route {1+route}: \n")
        with open('prtout.txt', 'a') as f:
            f.write(f"Route {1+route}: \n")
        for day, stops in enumerate(days, start=1):
            day_order = 1
            #print(f"\n Route: {1+route} Day: {day} \n")
            for stop in stops:            
                dataset.loc[stop, 'NEW BR'] = branch
                dataset.loc[stop, 'NEW RT'] = route
                subset_data.loc[stop, 'DAY'] = day
                dataset.loc[stop, 'NEW DAY'] = day
            
                subset_data.loc[stop, 'DAY ORDER'] = day_order
                dataset.loc[stop, 'NEW STOP'] = day_order
                #set dataset['DAY ORDER'] = day_order where dataset['CUST'] = subset_data.loc[stop, 'CUST']
                dataset.loc[dataset['CUST'] == subset_data.loc[stop, 'CUST'], 'DAY ORDER'] = day_order
                subset_data = dataset[dataset['Vehicle'] == route].copy()
                #print(f"""{day_order}: {subset_data.loc[stop, 'CUST NAME']} Sales: ${int(subset_data.loc[stop, 'Machine'])} Companion: ${int(subset_data.loc[stop, 'Companion'])} \n   {subset_data.loc[stop, 'ADDRESS']} """)
                #write to file prtout.txt
                
                with open('prtout.txt', 'a') as f:
                    f.write(f"""{day_order}: {subset_data.loc[stop, 'CUST NAME']} Sales: ${int(subset_data.loc[stop, 'Machine'])} Companion: ${int(subset_data.loc[stop, 'Companion'])} \n   {subset_data.loc[stop, 'ADDRESS']} """)
                day_order += 1

        # Update the original dataset with the new 'DAY' and 'DAY_ORDER' values for the current route
        dataset.update(subset_data)

    # Save the updated dataset
    dataset.to_csv('your_updated_dataset.csv')
# Define a function to calculate the distance between two nodes
def calculate_distance(node1, node2):
    lat1, lon1 = node1['Latitude'], node1['Longitude']
    lat2, lon2 = node2['Latitude'], node2['Longitude']
    return haversine((lat1, lon1), (lat2, lon2))
def set_path_for_each_route(dataset):
    # Get unique routes
    unique_routes = dataset['Vehicle'].unique()

    # Iterate over unique routes
    for route in unique_routes:
        # Subset your data for the current route
        subset_data = dataset[dataset['Vehicle'] == route]

        # Create a graph for this route
        graph = nx.Graph()

        # Add nodes to the graph
        for index, row in subset_data.iterrows():
            graph.add_node(index, Latitude=row['Latitude'], Longitude=row['Longitude'])

        # Add edges to the graph (complete graph, every node is connected to every other node)
        for node1, data1 in graph.nodes(data=True):
            for node2, data2 in graph.nodes(data=True):
                if node1 != node2:
                    distance = calculate_distance(data1, data2)
                    graph.add_edge(node1, node2, weight=distance)

        # Find the shortest path in the graph
        # Note: This assumes that the depot is the first node in the subset_data DataFrame
        depot = subset_data.index[0]
        path = nx.approximation.greedy_tsp(graph)#, source=depot)

        # Assign the order of the path to the 'DAY' column of the subset_data DataFrame
        for order, node in enumerate(path):
            subset_data.loc[node, 'DAY'] = order

        # Update the original dataset with the new 'DAY' values for the current route
        dataset.update(subset_data)
    dataset['ROUTE'] = dataset['Vehicle']
    # Save the updated dataset
    dataset.to_csv('your_updated_dataset.csv')

def haversinedm(lat1: float, lon1: float, lat2: float, lon2: float, unit: str = 'kilometers') -> float:
    """
    Calculate the distance between two points on the Earth's surface using the Haversine formula.

    Parameters:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
        unit (str): Unit of the distance result. Default is 'kilometers'.

    Returns:
        distance (float): The distance between the two points in the specified unit.
    """
    # Check if the latitude and longitude values are within valid ranges
    if not (-90 <= lat1 <= 90) or not (-90 <= lat2 <= 90) or not (-180 <= lon1 <= 180) or not (-180 <= lon2 <= 180):
        raise ValueError("Invalid latitude or longitude values")

    # Convert the latitude and longitude values from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Calculate the differences in latitude and longitude in radians
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Define helper functions to calculate the intermediate values 'a' and 'c' in the Haversine formula
    def calculate_a(dlat: float, dlon: float) -> float:
        return np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2

    def calculate_c(a: float) -> float:
        return 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate the intermediate values 'a' and 'c' using the helper functions
    a = calculate_a(dlat, dlon)
    c = calculate_c(a)

    # Calculate the distance using the Haversine formula and the Earth's radius 'R'
    distance = EARTH_RADIUS * c

    # Check if the unit is valid
    conversion_factors = {
        'miles': 0.621371,
        'kilometers': 1,
        'nautical miles': 0.539957,
    }
    if unit not in conversion_factors:
        raise ValueError("Invalid unit")

    # Convert the distance using the appropriate conversion factor based on the unit
    distance *= conversion_factors[unit]

    return distance
def calculate_a(dlat, lat1_in_radians, lat2_in_radians, dlon):
    return math.sin(dlat / 2)**2 + math.cos(lat1_in_radians) * math.cos(lat2_in_radians) * math.sin(dlon / 2)**2
def calculate_c(a):
    return 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
def print_solution(data, manager, routing, solution, dataset):
    total_distance = 0  # Total distance covered by all the vehicles
    total_load = 0  # Total load carried by all the vehicles
    total_load2 = 0  # Total load carried by all the vehicles
    previous_index = routing.Start(0)
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route  {}:\n'.format(vehicle_id)
        route_distance = 0  # Distance covered by the current vehicle
        route_load = 0  # Load carried by the current vehicle
        route_load2 = 0  # Load carried by the current vehicle
        route_count = 0  # Count of stops made by the current vehicle
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            #add the index row of dataset to plan_output
            plan_output += ' {} -> '.format(dataset.iloc[index]['CUST NAME'])
            plan_output += ' {}m\n'.format(routing.GetArcCostForVehicle(previous_index, index, vehicle_id))
            route_count += 1
            node_index = manager.IndexToNode(index)  #
            route_load += data['demands'][node_index]
            route_load2 += data['demands2'][node_index]
            data['Vehicle'][node_index] = vehicle_id
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route_load += data['demands'][node_index]
        route_load2 += data['demands2'][node_index]
        data['Vehicle'][node_index] = vehicle_id
        crint('Route {} has {} stops \n'.format(vehicle_id+1, route_count),forecolor="cyan",backcolor="navy",style="bold_italic",font_size=20)
        crint('Companion: ${:,.0f} \n'.format(route_load), indent=30,style="bold_italic")
        crint('Big 5: ${:,.0f}\n'.format(route_load2), indent=30,style="bold_italic")
        crint('Distance: {}m\n'.format(route_distance), indent=30,style="bold_italic",font_size=18)
        total_distance += route_distance
        total_load += route_load  # Add the load carried by the vehicle to the total load
        total_load2 += route_load2  # Add the load carried by the vehicle to the total load
    crint('Total Distance of all routes: {}m'.format(total_distance),forecolor="cyan",backcolor="navy",style="bold_italic",font_size=20)

def create_map(data, dataset):
    """
    Generates a map with markers and polygons to visualize data points and clusters.

    Args:
        data (dict): A dictionary containing the 'Vehicle' key with a list of vehicle IDs.
        dataset (DataFrame): A pandas DataFrame containing the latitude, longitude, companion, and machine data for each data point.

    Returns:
        m (folium.Map): The generated map with markers and polygons representing the data points and clusters.
    """
    dataset_cp = dataset.copy()
    dataset_cp = dataset_cp[['Latitude', 'Longitude', 'Companion', 'Machine']]
    # add a column to dataset_cp that has vehicle from data['Vehicle']
    dataset_cp['Vehicle'] = data['Vehicle']
    companion_total = dataset_cp.groupby('Vehicle')['Companion'].sum()
    machine_total = dataset_cp.groupby('Vehicle')['Machine'].sum()
    # Calculate the size of each cluster
    cluster_sizes = dataset_cp.groupby('Vehicle').size()
    #centers_df = pd.merge(centers_df, cluster_sizes.rename('Size'), left_on='Cluster', right_index=True)
    # centers_df['Size'] = centers_df['Cluster'].apply(lambda x: len(cust_numb_dict[x]))
    m = folium.Map(location=[dataset['Latitude'].mean(),
                dataset['Longitude'].mean()], zoom_start=8)
    # Generate a color map for each unique vehicle
    vehicle_ids = dataset_cp['Vehicle'].unique()
    colors = [f"#{''.join([np.random.choice(list('ABCDEF0123456789')) for j in range(6)])}" for i in range(
        len(vehicle_ids))]
    color_map = dict(zip(vehicle_ids, colors))
    for vehicle in vehicle_ids:
        # Extract the data for this vehicle
        vehicle_data = dataset_cp[dataset_cp['Vehicle'] == vehicle][['Latitude', 'Longitude']].values

        # Calculate the Convex Hull of the vehicle points
        if len(vehicle_data) > 2:  # ConvexHull requires at least 3 points
            hull = ConvexHull(vehicle_data)
            # Draw the Convex Hull as a polygon on the map
            folium.Polygon(locations=vehicle_data[hull.vertices].tolist(),
                        fill=True,
                        color=color_map[vehicle],
                        tooltip=Tooltip(f"Route: {vehicle+1}, Stop Count: {cluster_sizes[vehicle]}, Total Companion: {companion_total[vehicle]}, Big 5: {machine_total[vehicle]}")).add_to(m)
        # Draw the markers for the vehicle
        FastMarkerCluster(vehicle_data.tolist()).add_to(m)
    # Save and render the map
    m.save('map.html')
    m.render()
    # Display the map
    return m

import folium
from matplotlib import colors as mcolors
from folium import plugins
from folium.features import DivIcon

def get_good_map(dataset):
  dataset = dataset.sort_values(['Vehicle', 'DAY'])
  vehicles = dataset['Vehicle'].unique()

  # Create a Map instance
  m = folium.Map(location=[39.7, -76.5], zoom_start=8)
  # Create a color map
  colors = list(mcolors.CSS4_COLORS.keys())  # list of named colors
  color_map = {vehicle: colors[i+13] for i, vehicle in enumerate(vehicles)}
  # Create a layer for each route
  for vehicle in dataset['Vehicle'].unique():
      route = folium.FeatureGroup(name=f'Route {vehicle}')
      vehicle_data = dataset[dataset['Vehicle'] == vehicle]

      # Add points to the route
      for index, row in vehicle_data.iterrows():
          tooltip_text = f"""
          <table style="width:100%">
            <tr>
              <th>Name:</th>
              <td><a href="https://www.google.com/search?q={row['CUST NAME']}+{row['ADDRESS']}" target="_blank">{row['CUST NAME']}</a></td>
            </tr>
            <tr>
              <th>Index:</th>
              <td>{index}</td>
            </tr>
       
            <tr>
              <th>Address:</th>
              <td><a href="https://www.google.com/maps/dir//{row['ADDRESS']}" target="_blank">{row['ADDRESS']}</a></td>
            </tr>
            <tr>
              <th>Companion:</th>
              <td>${row['Companion']:,.2f}</td>
            </tr>
            <tr>
              <th>Sales:</th>
              <td>${row['Machine']:,.2f}0</td>
            </tr>
            <tr>
              <th>Day:</th>
              <td>{int(row['DAY'])}</td>
            </tr>
          </table>
          """
          iframe = folium.IFrame(html=tooltip_text, width=400, height=150)
          tooltip = folium.Tooltip(iframe)
          folium.CircleMarker(
              location=[row['Latitude'], row['Longitude']],
              popup=tooltip_text,
              tooltip=tooltip_text,
              color='#3186cc',
              fill=True,
              fill_color='#3186cc',
              radius = 7,
              weight = 1
          ).add_to(route)

      # Add lines to the route
      locations = vehicle_data[['Latitude', 'Longitude']].values.tolist()
      folium.Polygon(locations=locations, color=color_map[vehicle], fill=True, fill_opacity=0.3, weight=2).add_to(route)

      # Add the route to the map
      route.add_to(m)
  plugins.Fullscreen(
      position="topleft",
      title="Expand me",
      title_cancel="Exit me",
      force_separate_button=True,
  ).add_to(m)

  # Add a layer control to the map
  folium.LayerControl().add_to(m)

  m.save("goodmap.html")
  # Display the map
  return m

def crint(string_to_print, forecolor='black', backcolor='white', style='normal', indent=0, font_size=17):

    """
    Custom print function to display colored text in a Jupyter Notebook using HTML.
    Parameters:
        string_to_print (str): The string to print.
        forecolor (str): The text color.
        backcolor (str): The background color.
        style (str): The text style ('normal', 'bold', 'italic').
        indent (int): The left margin indentation in pixels.
    """
    style_mapping = {
        'normal': 'font-style: normal; font-weight: normal; text-decoration: none;',
        'bold': 'font-style: normal; font-weight: bold; text-decoration: none;',
        'italic': 'font-style: italic; font-weight: normal; text-decoration: none;',
        'bold_italic': 'font-style: italic; font-weight: bold; text-decoration: none;',
        'underline': 'font-style: normal; font-weight: normal; text-decoration: underline;',
        'strikethrough': 'font-style: normal; font-weight: normal; text-decoration: line-through;',
        'overline': 'font-style: normal; font-weight: normal; text-decoration: overline;'
    }

    
    css_style = f"color: {forecolor}; background-color: {backcolor}; {style_mapping.get(style, 'font-style: normal; font-weight: normal; text-decoration: none;')} margin-left: {indent}px; font-size: {font_size}px"
    display(HTML(f"<span style='{css_style}'>{string_to_print}</span>"))

#imports to perform a wait
import time 
import sys
from IPython.display import clear_output
def print_days_per_route(dataset):
    i = 0
    for cunt in dataset.groupby('NEW RT')['NEW DAY'].nunique():
        i += 1
        crint(f"Route {i} Days: {cunt}",'cyan', 'navy')
def write_solution(dataset):
    now = datetime.datetime.now()
    with open(f'routewithdays {now.day}-{now.month}-{now.year} {now.hour}{now.minute}.csv','a') as ff:
        writer = csv.writer(ff)
        writer.writerow(["Route", "Route Day", "Stop", "Name", "Address", "Big 5", "Companion", "Distance"])
        for vehicle_id in dataset['Vehicle'].unique():
            c_stop = 0
            # Loop through all rows in dataset where dataset['Vehicle'] == vehicle_id
            subset_data = dataset[dataset['Vehicle'] == vehicle_id]
            for index, row in subset_data.iterrows():
                c_route = vehicle_id + 1
                c_stop += 1
                c_name = row['CUST NAME']
                c_addy = row['ADDRESS']
                c_big = row['Machine']
                c_comp = row['Companion']
                c_route_day = row['DAY']
                writer.writerow([c_route, c_route_day, c_stop, c_name, c_addy, c_big, c_comp])

In [3]:
import pandas as pd
import numpy as np

file_path = "./deset.xls"
# load reroute.xls into dataset, set the column headers to the first row of the dataset and drop the second row
dataset = pd.read_excel(file_path)
dataset = dataset.iloc[1:, 0:]
dataset = dataset.reset_index(drop=True)
#drop the first column of dataset
dataset = dataset.iloc[:, 1:]
# drop the first column of dataset
dataset = dataset.iloc[:, 1:]

# if dataset does not have a column named 'Latitude' then create it
if 'Latitude' not in dataset.columns:
    dataset['Latitude'] = np.nan

# if dataset does not have a column named 'Longitude' then create it  
if 'Longitude' not in dataset.columns:
    dataset['Longitude'] = np.nan

# split the 'ADDRESS' column by comma and keep only the first part
dataset['ADDRESS'] = dataset['ADDRESS'].str.split(',').str[0]

In [4]:
import os
from tqdm import tqdm
if not os.path.exists('dataset.csv'):
    geolocator = Nominatim(user_agent="my-stuff", timeout=10)
    for i in tqdm(range(len(dataset))):
        #clear search_address
        search_address = ''
        #if dataset['Latitude'][i] doees not have a value
        if math.isnan(dataset['Latitude'][i]):
            short_zip = str(dataset['ZIP'][i])
            search_address = f"{dataset['HOUSE NUMBER'][i]} {dataset['ADDRESS'][i]}, {dataset['CITY'][i]}, {short_zip[:5]}"
            location = geolocator.geocode(search_address)   # geocode method returns an object of type location
            if location is not None:
                dataset.loc[i, 'Latitude'] = location.latitude
                dataset.loc[i, 'Longitude'] = location.longitude
            else:
                search_address = f"{dataset['CITY'][i]} {short_zip[:5]} "
                location = geolocator.geocode(search_address)   # geocode method returns an object of type location
                if location is not None:
                    dataset.loc[i, 'Latitude'] = location.latitude
                    dataset.loc[i, 'Longitude'] = location.longitude
                else:
                    search_address = short_zip[:5]               
                    if location is not None:
                        dataset.loc[i, 'Latitude'] = location.latitude
                        dataset.loc[i, 'Longitude'] = location.longitude
                    else:
                        crint("waiting...", "green", "on_red")
                        crint(f"No location found for {search_address}")
                        crint("Yikes try only Long ZIp CodE error")
#save dataset to csv
    print("saving.....")
    dataset.to_csv('dataset.csv', index=False)  
else:
    print("reading.......")
    dataset = pd.read_csv('dataset.csv')


reading.......


In [5]:


def clean_data(dataset):
    # Ensure 'CUST' column is of consistent data type
    dataset['CUST'] = dataset['CUST'].astype(str)
    
    # Find the rows where the customer is the one in question and replace their coordinates
    dataset.loc[dataset['CUST'] == "216500035", 'Longitude'] = -76.376
    dataset.loc[dataset['CUST'] == "216500035", 'Latitude'] = 39.511
    
    return dataset

dataset = clean_data(dataset.copy())





In [6]:
if True: #not os.path.exists('distance_matrix.csv'):
    #create a new distance matrix with the new latitudes and longitudes
    new_distance_matrix = pd.DataFrame(index=dataset.index, columns=dataset.index)
    new_distance_matrix.head()
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            new_distance_matrix.iloc[i, j] = haversinedm(dataset['Latitude'][i], dataset['Longitude'][i], dataset['Latitude'][j], dataset['Longitude'][j])
    distance_matrix = new_distance_matrix

else:
    distance_matrix = pd.read_csv('distance_matrix.csv')

#distance_matrix = distance_matrix.astype(int)
distance_matrix = distance_matrix.add(.9999)
distance_matrix = distance_matrix.astype(int)
latlong_df = dataset[['Latitude', 'Longitude']]
distance_matrix.to_csv('distance_matrix.csv', index=False)


# 3. Problem Solving

In this section, we solve the optimization problem and obtain the optimal set of routes.

We define the search parameters for the solver, including the search strategy and the time limit. We use the guided local search metaheuristic, which is a good choice for vehicle routing problems.

We then call the `SolveWithParameters` method of the routing model to solve the problem. This method returns a solution, which is an object that contains the optimal set of routes.


## Create Data Model

In [7]:
def create_data_model(dataset, distance_matrix, vehicle_capacities, vehicle_capacities2):
    latitudes = dataset['Latitude'].tolist()
    longitudes = dataset['Longitude'].tolist()
    # Extract demands
    data = {}
    demands = dataset['Companion'].tolist()
    demands2 = dataset['Machine'].tolist()
    data['distance_matrix'] = distance_matrix.values.tolist()
    data['demands'] = [int(demand) for demand in demands]
    data['demands2'] = [int(demand) for demand in demands2]
    data['vehicle_capacities'] = [int(capacity) for capacity in vehicle_capacities]
    data['vehicle_capacities2'] = [int(capacity) for capacity in vehicle_capacities2]
    data['num_vehicles'] = 3
    data['depot'] = 0
    data['starts'] =  [88, 200, 195]
    data['ends'] = [365, 309, 294]
    data['Vehicle'] = [0] * len(data['demands'])
    return data


In [8]:
#rename BIG 5 AVG to Machine and COMP AVG to Companion
#dataset = dataset.rename(columns={'BIG 5 AVG': 'Machine', 'COMP AVG': 'Companion'})
prev_obj, best_obj = 999999999, 999999999
try:
    dataset['Machine'] =dataset['BIG 5 AVG'] 
    dataset['Companion']=dataset['COMP AVG'] 
except Exception as e:
    print(e)

## Run the ReRoute Solver

In [9]:
vehicle_capacities = [8000, 19000, 12000] #companion max taget for each route
vehicle_capacities2 = [50000, 54000,55000] #big 5 target max for each route
data = create_data_model(dataset, distance_matrix, vehicle_capacities, vehicle_capacities2)
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['starts'], data['ends'])
print_starts_ends(data,dataset)
routing_parameters = pywrapcp.DefaultRoutingModelParameters()
#routing_parameters.solver_parameters.trace_propagation = True
#routing_parameters.solver_parameters.trace_search = True
#routing = pywrapcp.RoutingModel(manager, routing_parameters)
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager) #without routing parameters
# Create the distance callback, which takes two arguments (the from and to node indices)
def create_distance_callback(data, manager, vehicle):
    _distances = data['distance_matrix']
    _starts = data['starts']
    _ends = data['ends']
    def distance_callback(from_index, to_index):
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        start_node = _starts[vehicle]
        end_node = _ends[vehicle]
        _home_cost =  _distances[start_node][to_node]
        _end_cost = _distances[from_node][end_node]
        to_cost =_distances[from_node][to_node]
        off_track_factor = (int(((_home_cost +_end_cost)/2)/2))
        _arc_cost = int(to_cost*off_track_factor)
        output = f"Vehicle: {vehicle}, From: {from_node}, To: {to_node},End Cost: {_end_cost},Home Cost: {_home_cost}, To Cost: {to_cost}, Factor {off_track_factor},Arc Cost: {_arc_cost}"
        #print(output)
        return _arc_cost
    return distance_callback
for vehicle_id in range(data['num_vehicles']):
    distance_callback = create_distance_callback(data, manager, vehicle_id)
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfVehicle(distance_callback_index, vehicle_id)
def demand_callback(from_index): return data['demands'][manager.IndexToNode(from_index)]
def demand_callback2(from_index):
    return data['demands2'][manager.IndexToNode(from_index)]
# Create and register a transit callback.
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
demand_callback_index2 = routing.RegisterUnaryTransitCallback(demand_callback2)
#routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
max_distance = 30000
routing.AddDimension(distance_callback_index, 0,  # no slack
    max_distance, True,  # start cumul to zero
    'Distance')
routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')
routing.AddDimensionWithVehicleCapacity(demand_callback_index2, 0,  # null capacity slack
    data['vehicle_capacities2'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity2')
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.solution_limit =999
#365 solutions is good
search_parameters.use_full_propagation = True
search_parameters.time_limit.seconds = 91 # ComputeSeconds
#search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION)
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.lns_time_limit.seconds =10
search_parameters.log_search = True
print("data: starting to solve")
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(data, manager, routing, solution, dataset)
    dataset['Vehicle'] = data['Vehicle']
    #CONVERT THE VALUES IN dataset['DAY] TO integers
    #dataset['DAY'] = dataset['DAY'].astype(int)
    set_path_for_each_route(dataset)
    set_days_for_each_route(dataset,5,9)
    prev_obj, best_obj = best_tracker(solution, prev_obj, best_obj)
    print_days_per_route(dataset)
    map_df = dataset
    m=get_good_map(map_df)
    write_ouput_files(dataset)
    
else:
    print("The solver failed to find a solution.")
m

data: starting to solve


I0000 00:00:1697778096.882022  136119 search.cc:276] Start search (memory used = 353.50 MB)
I0000 00:00:1697778097.787918  136119 search.cc:276] Root node processed (time = 905 ms, constraints = 5695, memory used = 353.50 MB)
I0000 00:00:1697778098.082522  136119 search.cc:276] Solution #0 (56550, time = 1200 ms, branches = 34, failures = 1, depth = 33, memory used = 354.85 MB, limit = 1%)
I0000 00:00:1697778098.104318  136119 search.cc:276] Solution #1 (56520, maximum = 56550, time = 1222 ms, branches = 38, failures = 3, depth = 33, Relocate<1>, neighbors = 248, filtered neighbors = 1, accepted neighbors = 1, memory used = 355.21 MB, limit = 1%)
I0000 00:00:1697778098.129401  136119 search.cc:276] Solution #2 (56497, maximum = 56550, time = 1247 ms, branches = 43, failures = 5, depth = 33, Relocate<1>, neighbors = 1584, filtered neighbors = 2, accepted neighbors = 2, memory used = 355.21 MB, limit = 1%)
I0000 00:00:1697778098.151499  136119 search.cc:276] Solution #3 (56490, maximum =

# DATASET
#

In [ ]:
# Assuming `routing` is your RoutingModel instance and `solution` is the solution object
if solution:
    print(f"Objective: {solution.ObjectiveValue()}")
    for vehicle_id in range(data['num_vehicles']):

            print(f"Route for vehicle {vehicle_id} (Objective value: {solution.RouteObjectiveValue(vehicle_id)}):")
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                route.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(routing.End(vehicle_id)))
            print(" -> ".join(map(str, route)))


Objective: 36742


AttributeError: 'Assignment' object has no attribute 'RouteObjectiveValue'

## Write the files